<div class="alert alert-block alert-info">

Passed/approved.
    
See comments in ex 1 q2 and ex2 q1.

In [93]:
#Import packages
import pandas as pd
import numpy as np

import statsmodels.formula.api as smf

## Question 1
1. Prediction problem: aim to predict how many people will get ill using information at hand, and we are less interested in why these cases occur - meaning we don’t really care where effect is coming from.


2. Causal inference problem: we are interested in what happens if we change the value of a certain variable (eg changing the amount spent on advertising) on the potential profits


3. Causal inference: seeing the effect of including a picture in a tweet, while keeping all else is equal. We are interested in answering which factors will maximize the number of retweets, which makes this a causal inference problem. 


4. Predict who is at risk of being/becoming a drug addict- we don’t care which characteristics cause people to have addiction problems. We just want to target them with an intervention program.
 
5. Causal inference: we know the facts, and are interested in the counterfactual outcome of what would happen if someone would be placed somewhere else and how this would have an impact on the outcome



In [94]:
#Read data
halloween = pd.read_csv('halloween.csv')
df1 = pd.read_table("lab_mark_train1.dat", sep="\t")
df2 = pd.read_table('lab_mark_train2.dat', sep = '\t')

## Question 2
*a) What is the population of interest that you are sampling from?*

Children in New Haven, CT - from class discussion: All american kids (may not be representative for all kids in america, but maybe only representative for kids in New haven) 

*What is the definition of Yi, Y1i, Y0i, Di and Xi?*

Yi = Choosing fruits or not - binary variable 

Y1i = Chooses fruit - binary outcome if the person i is subject to treatment Di=1

Y0i = Chooses candy - binary outcome for not subject to treatment 

Di = Obama or not - if the person i saw the picture of Obama before choosing fruit or candy (also binary)

Xi = Age of kid (control variable)


*In words, what does it mean if Y12 = 0 in these data?*

Child number 12 choses candy

*In words, what does it mean if D58 = 1 in these data?*

Child number 58 saw picture of Obama

*b) Let j = 22 and k = 9. Open the data and compute (find) the following if you can: Yj, Dj, Y1j, Dk, Y0k, Xk*


<div class="alert alert-block alert-info">

$Y_{1_i}$ is the potential outcome for a treated observation (so we do not know whether the kid chose the fruit or not based on this). Likewise $Y_{0_i}$ is the potential outcome for an untreated observation - so again, this could both be taking or not taking the fruit for kid i if kid i is in the control group. See solution sheet.

In [95]:
print(halloween.iloc[21])
print('\n')
print(halloween.iloc[8])

obama     0.0
fruit     0.0
age      11.0
Name: 21, dtype: float64


obama    0.0
fruit    0.0
age      6.0
Name: 8, dtype: float64


*Answer question b):*

Yj= Did not choose fruit 

Dj= Did not see Obama

Y1j= Not observed

Dk= Did not see Obama

Y0k= Was observed

Xk= 8


<div class="alert alert-block alert-info">

Yes, $Y_{0_{9}}$ is observed and thereby there is also an outcome that we can report; $Y_{0_{9}} = 0$ so this child was in the control group and did not take fruit. See solution sheet.  

*c) Comparison of averages*

In [96]:
print(halloween[halloween['obama'] == 1].describe())
print(halloween[halloween['obama'] == 0].describe())

print('\n')

difference = halloween[halloween['obama'] == 1].mean() - halloween[halloween['obama'] == 0].mean()

print('Mean difference: ', difference['fruit'])


       obama       fruit         age
count  444.0  444.000000  444.000000
mean     1.0    0.274775    8.515766
std      0.0    0.446904    3.131753
min      1.0    0.000000    3.000000
25%      1.0    0.000000    6.000000
50%      1.0    0.000000    8.000000
75%      1.0    1.000000   11.000000
max      1.0    1.000000   19.000000
       obama       fruit         age
count  778.0  778.000000  778.000000
mean     0.0    0.237789    8.517995
std      0.0    0.426003    3.062046
min      0.0    0.000000    2.000000
25%      0.0    0.000000    6.000000
50%      0.0    0.000000    8.000000
75%      0.0    0.000000   11.000000
max      0.0    1.000000   19.000000


Mean difference:  0.036985571689941904


Out of the 444 who saw Obama 27% chose a fruit, Out of the 778 who didn’t see Obama only 23% chose a fruit. The mean difference between the treatment groups is 0.037.

*d) Simple regression model with robust standard error*

*e) compute confidence interval*

In [97]:
#Fit regression model to data
model = smf.ols(formula = 'fruit ~obama', data = halloween)
results = model.fit(cov_type = 'HC0')


#Calculate confidence interval 
print(results.conf_int())
results.summary()



                  0         1
Intercept  0.207874  0.267704
obama     -0.014191  0.088162


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  fruit   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     2.006
Date:                Thu, 11 Feb 2021   Prob (F-statistic):              0.157
Time:                        12:42:49   Log-Likelihood:                -712.11
No. Observations:                1222   AIC:                             1428.
Df Residuals:                    1220   BIC:                             1438.
Df Model:                           1                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.2378      0.015     15.579      0.000       0.208       0.268
obama          0.0370      0.026      1.416      0.157      -0.014       0.088
==============================================================================
Omnibus:                      236.213   Durbin-Watson:                   2.084
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              290.347
Skew:                           1.144   Prob(JB):                     8.95e-64
Kurtosis:                       2.319   Cond. No.                         2.42
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

What we can see is that there is no significant effect of seeing Obama on chosing fruits over candies. The result from our means comparison is the same as aour regression, because we are dealing with binary variables. 

*f) Average age comparison*

In [98]:
treat = halloween[halloween['obama']==1]
control = halloween[halloween['obama'] == 0]

print(treat['age'].mean())
print(control['age'].mean())
print('Mean difference: ', treat['age'].mean() - control['age'].mean() )

8.515765765765765
8.517994858611825
Mean difference:  -0.002229092846059544


* g) average age comparison using linear regression

In [101]:
#Fit regression model to data
model1 = smf.ols(formula = 'age ~ obama', data = halloween)
results1 = model1.fit(cov_type = 'HC0')

#Confidence intervals
CI = results1.conf_int(alpha=0.05)
print(CI.loc['obama'])

results1.summary()



0   -0.364033
1    0.359575
Name: obama, dtype: float64


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    age   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                 0.0001458
Date:                Thu, 11 Feb 2021   Prob (F-statistic):              0.990
Time:                        12:44:58   Log-Likelihood:                -3110.6
No. Observations:                1222   AIC:                             6225.
Df Residuals:                    1220   BIC:                             6235.
Df Model:                           1                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      8.5180      0.110     77.642      0.000       8.303       8.733
obama         -0.0022      0.185     -0.012      0.990      -0.364       0.360
==============================================================================
Omnibus:                       47.467   Durbin-Watson:                   2.070
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               52.419
Skew:                           0.505   Prob(JB):                     4.14e-12
Kurtosis:                       2.895   Cond. No.                         2.42
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

*h) Does randomization hold?*

The mean approaches zero, and the confidence interval includes 0, which supports our assumption that the treatment condition was indeed randomly assigned, regardless of the childs age. 

## Question 3
*a)*

Education is not random: it's dependent on your socio-economic background

*b)*

Would also not assume this is random: showing up early might indicate that these students are already more motivated than those that show up late

*c)*

It seems like assignement to which weekday you can do yoga was random, so in this case, as-good-as random assignment might hold

*d)*

It's probably not random that the railway station opened in that particular neighbourhood: maybe it was part of a larger plan to make that neighbourhood more attractive etc. There is also the risk of a spill over effect if we compare the effect to a nearby neighbourhood, who after all could also use this new station.

# Second exercise sheet

## Question 1

In [9]:
df1.head()

,training,age,yrs_educ,black,hispanic,married,nodegree,earnings1975,earnings1978
0,0,33,12,0,0,1,0,11384.66,23136.76
1,0,21,12,0,0,0,0,11860.89,1111.25
2,0,26,10,0,0,0,1,13072.94,15445.20
3,0,36,12,0,0,1,0,19303.26,25564.67
4,0,40,8,0,0,1,1,15719.03,13973.38


In [10]:
print('Mean 1975: ', df1['earnings1975'].mean())
print('STD 1975: ', df1['earnings1975'].std())
print('Mean 1978: ', df1['earnings1978'].mean())
print('STD 1978: ', df1['earnings1978'].mean())

Mean 1975:  13457.810715919513
STD 1975:  9317.233096061858
Mean 1978:  14684.925903544263
STD 1978:  14684.925903544263


There is a mean difference between the two groups, which indicates that during these three years, something happened that increased the average earnings of the sample. However, the standard deviation indicates that there might be a big difference/effect for a subgroup in the data, because the standard deviation also became bigger (indicating a wider dispersion of the sample)

*b)*

population of interest: Unemployed Americans/disadvantaged workers 

Yi = earnings, Di = training

Estimand: Average Treatment Effect


<div class="alert alert-block alert-info">

Yes, but note that only men with regard to the population.

*c) Regression with 90% confidence interval*

In [69]:
#fit model
model3 = smf.ols(formula = 'earnings1978 ~training', data = df1)
results3 = model3.fit(cov_type = 'HC0')

#Print confidence intervals
print('90% CI: ', results3.conf_int(alpha = 0.1).loc['training'])
results3.summary()

90% CI:  0   -9541.856709
1   -8198.758571
Name: training, dtype: float64


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           earnings1978   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     472.0
Date:                Wed, 10 Feb 2021   Prob (F-statistic):          3.36e-103
Time:                        16:10:58   Log-Likelihood:            -1.7248e+05
No. Observations:               16289   AIC:                         3.450e+05
Df Residuals:                   16287   BIC:                         3.450e+05
Df Model:                           1                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.485e+04     76.286    194.618      0.000    1.47e+04     1.5e+04
training   -8870.3076    408.273    -21.726      0.000   -9670.508   -8070.108
==============================================================================
Omnibus:                   156828.121   Durbin-Watson:                   1.969
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1446.062
Skew:                          -0.308   Prob(JB):                         0.00
Kurtosis:                       1.677   Cond. No.                         7.48
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

We can see that people who got training on average has a lower income than the average workforce. This effect is statistically significant. 

*d) Check covariates to see if that might explain the result*

In [84]:
age = smf.ols(formula = 'age ~training', data = df1)
age =age.fit(cov_type = 'HC0')

age.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    age   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     469.0
Date:                Wed, 10 Feb 2021   Prob (F-statistic):          1.46e-102
Time:                        16:26:08   Log-Likelihood:                -62144.
No. Observations:               16289   AIC:                         1.243e+05
Df Residuals:                   16287   BIC:                         1.243e+05
Df Model:                           1                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     33.2252      0.087    380.416      0.000      33.054      33.396
training      -8.5990      0.397    -21.657      0.000      -9.377      -7.821
==============================================================================
Omnibus:                     3694.982   Durbin-Watson:                   1.987
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              992.205
Skew:                           0.353   Prob(JB):                    3.51e-216
Kurtosis:                       2.019   Cond. No.                         7.48
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

In [71]:
education = smf.ols(formula = 'yrs_educ ~training', data = df1)
educ_res =education.fit(cov_type = 'HC0')

educ_res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               yrs_educ   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     233.8
Date:                Wed, 10 Feb 2021   Prob (F-statistic):           2.04e-52
Time:                        16:12:17   Log-Likelihood:                -40201.
No. Observations:               16289   AIC:                         8.041e+04
Df Residuals:                   16287   BIC:                         8.042e+04
Df Model:                           1                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     12.0275      0.023    529.823      0.000      11.983      12.072
training      -1.6470      0.108    -15.291      0.000      -1.858      -1.436
==============================================================================
Omnibus:                      916.160   Durbin-Watson:                   2.012
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1691.455
Skew:                          -0.426   Prob(JB):                         0.00
Kurtosis:                       4.329   Cond. No.                         7.48
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

In [72]:
black = smf.ols(formula = 'black~training', data = df1)
black_res =black.fit(cov_type = 'HC0')

black_res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  black   R-squared:                       0.120
Model:                            OLS   Adj. R-squared:                  0.120
Method:                 Least Squares   F-statistic:                     980.5
Date:                Wed, 10 Feb 2021   Prob (F-statistic):          4.71e-209
Time:                        16:12:34   Log-Likelihood:                -1430.2
No. Observations:               16289   AIC:                             2864.
Df Residuals:                   16287   BIC:                             2880.
Df Model:                           1                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0735      0.002     35.628      0.000       0.069       0.078
training       0.7278      0.023     31.313      0.000       0.682       0.773
==============================================================================
Omnibus:                     9871.375   Durbin-Watson:                   2.017
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            70478.106
Skew:                           2.999   Prob(JB):                         0.00
Kurtosis:                      11.237   Cond. No.                         7.48
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

In [73]:
hispanic = smf.ols(formula = 'hispanic~training', data = df1)
hispanic_res =hispanic.fit(cov_type = 'HC0')

hispanic_res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               hispanic   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.696
Date:                Wed, 10 Feb 2021   Prob (F-statistic):              0.193
Time:                        16:12:44   Log-Likelihood:                -1120.4
No. Observations:               16289   AIC:                             2245.
Df Residuals:                   16287   BIC:                             2260.
Df Model:                           1                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0720      0.002     35.234      0.000       0.068       0.076
training       0.0222      0.017      1.302      0.193      -0.011       0.056
==============================================================================
Omnibus:                    10675.514   Durbin-Watson:                   1.995
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            83054.081
Skew:                           3.298   Prob(JB):                         0.00
Kurtosis:                      11.880   Cond. No.                         7.48
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

In [74]:
married = smf.ols(formula = 'married~training', data = df1)
married_res =married.fit(cov_type = 'HC0')

married_res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                married   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     609.7
Date:                Wed, 10 Feb 2021   Prob (F-statistic):          3.39e-132
Time:                        16:12:56   Log-Likelihood:                -10166.
No. Observations:               16289   AIC:                         2.034e+04
Df Residuals:                   16287   BIC:                         2.035e+04
Df Model:                           1                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.7117      0.004    198.706      0.000       0.705       0.719
training      -0.5434      0.022    -24.693      0.000      -0.587      -0.500
==============================================================================
Omnibus:                     8489.227   Durbin-Watson:                   2.006
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3061.609
Skew:                          -0.907   Prob(JB):                         0.00
Kurtosis:                       1.897   Cond. No.                         7.48
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

In [76]:
nodegree = smf.ols(formula = 'nodegree~training', data = df1)
nodegree_res =nodegree.fit(cov_type = 'HC0')

nodegree_res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               nodegree   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     279.8
Date:                Wed, 10 Feb 2021   Prob (F-statistic):           2.74e-62
Time:                        16:14:16   Log-Likelihood:                -10329.
No. Observations:               16289   AIC:                         2.066e+04
Df Residuals:                   16287   BIC:                         2.068e+04
Df Model:                           1                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.2958      0.004     81.967      0.000       0.289       0.303
training       0.4348      0.026     16.727      0.000       0.384       0.486
==============================================================================
Omnibus:                    15174.875   Durbin-Watson:                   2.006
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2987.396
Skew:                           0.862   Prob(JB):                         0.00
Kurtosis:                       1.805   Cond. No.                         7.48
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

We can see that the group that received treatment differed from the control group (in this case the average population) in many ways. They were less educated, more likely to be black and not have a degree, to mention a few. The only factor which seems to be balance between the samples is the amount of people who self-report as being hispanic. The differences between the groups makes the coefficient from our regression less reliable, because the negative effect of the training observed is most likely because of selection bias, and not because of the treatment itself. 

*e) Covariate balance in new dataset, both by calculating mean differences and 90% confidence intervals as well as regressions, because, why not.*

In [45]:
import math


def results (name):
    
    treatment = df2[df2['training'] == 1]

    control = df2[df2['training'] == 0]

    treat_mean = treatment[name].mean()
    control_mean = control[name].mean()

    print(treat_mean, control_mean)
    diff = treat_mean - control_mean
    print('Mean: ', diff)

    print('90% CI:')
    #calculate marigin of error
    me = math.sqrt((treatment[name].std()**2/len(treatment[name]))+ (control[name].std()**2/len(control[name])))*1.645
    print(diff + me, diff - me)
    
    return


In [46]:
results('age')

24.626262626262626 24.44705882352941
Mean:  0.17920380273321612
90% CI:
1.0061721856172468 -0.6477645801508144


In [47]:
results('yrs_educ')

10.38047138047138 10.188235294117646
Mean:  0.19223608635373424
90% CI:
0.40853878146373124 -0.024066608756262736


In [48]:
results('black')

0.8013468013468014 0.8
Mean:  0.0013468013468013185
90% CI:
0.051110781167759446 -0.04841717847415681


In [49]:
results('hispanic')

0.09427609427609428 0.11294117647058824
Mean:  -0.018665082194493962
90% CI:
0.01901798062017382 -0.05634814500916174


In [50]:
results('married')

0.16835016835016836 0.15764705882352942
Mean:  0.010703109526638943
90% CI:
0.05682756618431835 -0.03542134713104046


In [51]:
results('nodegree')

0.7306397306397306 0.8141176470588235
Mean:  -0.08347791641909286
90% CI:
-0.030894662442647797 -0.13606117039553792


In [83]:
age = smf.ols(formula = 'age ~training', data = df2)
age = age.fit(cov_type = 'HC0')

print('90% CI: ', age.conf_int(alpha = 0.1).loc['training'])

age.summary()

90% CI:  0   -0.646468
1    1.004875
Name: training, dtype: float64


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    age   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.1274
Date:                Wed, 10 Feb 2021   Prob (F-statistic):              0.721
Time:                        16:25:32   Log-Likelihood:                -2389.2
No. Observations:                 722   AIC:                             4782.
Df Residuals:                     720   BIC:                             4792.
Df Model:                           1                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     24.4471      0.319     76.565      0.000      23.821      25.073
training       0.1792      0.502      0.357      0.721      -0.805       1.163
==============================================================================
Omnibus:                      185.949   Durbin-Watson:                   2.053
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              401.639
Skew:                           1.406   Prob(JB):                     6.10e-88
Kurtosis:                       5.332   Cond. No.                         2.46
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

In [86]:
education = smf.ols(formula = 'yrs_educ~training', data = df2)
education = education.fit(cov_type = 'HC0')

print('90% CI: ', education.conf_int(alpha = 0.1).loc['training'])

education.summary()

90% CI:  0   -0.023722
1    0.408194
Name: training, dtype: float64


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               yrs_educ   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     2.144
Date:                Wed, 10 Feb 2021   Prob (F-statistic):              0.144
Time:                        16:28:41   Log-Likelihood:                -1408.0
No. Observations:                 722   AIC:                             2820.
Df Residuals:                     720   BIC:                             2829.
Df Model:                           1                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     10.1882      0.078    129.910      0.000      10.035      10.342
training       0.1922      0.131      1.464      0.143      -0.065       0.450
==============================================================================
Omnibus:                       81.926   Durbin-Watson:                   2.082
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              149.631
Skew:                          -0.710   Prob(JB):                     3.22e-33
Kurtosis:                       4.719   Cond. No.                         2.46
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

In [87]:
black = smf.ols(formula = 'black ~training', data = df2)
black = black.fit(cov_type = 'HC0')

print('90% CI: ', age.conf_int(alpha = 0.1).loc['training'])

black.summary()

90% CI:  0   -9.252072
1   -7.945878
Name: training, dtype: float64


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  black   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                  0.001988
Date:                Wed, 10 Feb 2021   Prob (F-statistic):              0.964
Time:                        16:29:59   Log-Likelihood:                -362.16
No. Observations:                 722   AIC:                             728.3
Df Residuals:                     720   BIC:                             737.5
Df Model:                           1                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.8000      0.019     41.231      0.000       0.762       0.838
training       0.0013      0.030      0.045      0.964      -0.058       0.061
==============================================================================
Omnibus:                      157.587   Durbin-Watson:                   2.047
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              274.396
Skew:                          -1.504   Prob(JB):                     2.60e-60
Kurtosis:                       3.263   Cond. No.                         2.46
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

In [88]:
hispanic = smf.ols(formula = 'hispanic ~training', data = df2)
hispanic = hispanic.fit(cov_type = 'HC0')

print('90% CI: ', hispanic.conf_int(alpha = 0.1).loc['training'])

hispanic.summary()

90% CI:  0   -0.05629
1    0.01896
Name: training, dtype: float64


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               hispanic   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.6658
Date:                Wed, 10 Feb 2021   Prob (F-statistic):              0.415
Time:                        16:30:35   Log-Likelihood:                -171.28
No. Observations:                 722   AIC:                             346.6
Df Residuals:                     720   BIC:                             355.7
Df Model:                           1                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1129      0.015      7.356      0.000       0.083       0.143
training      -0.0187      0.023     -0.816      0.415      -0.063       0.026
==============================================================================
Omnibus:                      370.258   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1433.283
Skew:                           2.569   Prob(JB):                         0.00
Kurtosis:                       7.609   Cond. No.                         2.46
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

In [89]:
married = smf.ols(formula = 'married ~training', data = df2)
married = married.fit(cov_type = 'HC0')

print('90% CI: ', married.conf_int(alpha = 0.1).loc['training'])

married.summary()

90% CI:  0   -0.035349
1    0.056755
Name: training, dtype: float64


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                married   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.1461
Date:                Wed, 10 Feb 2021   Prob (F-statistic):              0.702
Time:                        16:31:01   Log-Likelihood:                -303.61
No. Observations:                 722   AIC:                             611.2
Df Residuals:                     720   BIC:                             620.4
Df Model:                           1                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1576      0.018      8.918      0.000       0.123       0.192
training       0.0107      0.028      0.382      0.702      -0.044       0.066
==============================================================================
Omnibus:                      222.219   Durbin-Watson:                   1.991
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              460.546
Skew:                           1.834   Prob(JB):                    9.86e-101
Kurtosis:                       4.364   Cond. No.                         2.46
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

In [91]:
nodegree = smf.ols(formula = 'nodegree ~training', data = df2)
nodegree = nodegree.fit(cov_type = 'HC0')

print('90% CI: ', nodegree.conf_int(alpha = 0.1).loc['training'])

nodegree.summary()

90% CI:  0   -0.135977
1   -0.030979
Name: training, dtype: float64


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               nodegree   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     6.841
Date:                Wed, 10 Feb 2021   Prob (F-statistic):            0.00910
Time:                        16:31:40   Log-Likelihood:                -384.88
No. Observations:                 722   AIC:                             773.8
Df Residuals:                     720   BIC:                             782.9
Df Model:                           1                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.8141      0.019     43.144      0.000       0.777       0.851
training      -0.0835      0.032     -2.615      0.009      -0.146      -0.021
==============================================================================
Omnibus:                      133.057   Durbin-Watson:                   2.059
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              213.979
Skew:                          -1.331   Prob(JB):                     3.43e-47
Kurtosis:                       2.829   Cond. No.                         2.46
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

*f) Fit model*

In [62]:
#fit model
model4 = smf.ols(formula = 'earnings1978 ~training', data = df2)
results4 = model4.fit(cov_type = 'HC0')

#Print confidence intervals
print(results4.conf_int(alpha = 0.1).loc['training'])
results4.summary()

0      84.500139
1    1688.107322
Name: training, dtype: float64


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           earnings1978   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     3.306
Date:                Wed, 10 Feb 2021   Prob (F-statistic):             0.0694
Time:                        15:04:06   Log-Likelihood:                -7333.1
No. Observations:                 722   AIC:                         1.467e+04
Df Residuals:                     720   BIC:                         1.468e+04
Df Model:                           1                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   5090.0483    277.042     18.373      0.000    4547.057    5633.040
training     886.3037    487.462      1.818      0.069     -69.104    1841.712
==============================================================================
Omnibus:                      384.449   Durbin-Watson:                   2.163
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3767.287
Skew:                           2.195   Prob(JB):                         0.00
Kurtosis:                      13.294   Cond. No.                         2.46
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

*f) continued and g)*

Comparing the two datasets, it seems like the second dataset shows that the training worked if we are satisfied with a 90% confidence level. The covariates of this dataset are balanced (so we can assume that the random assignment was effective) and there seems to be a positive effect of the training on the person's earnings at the 90% confidence level, an effect that does not hold at a 95% confidence level. This result varies a lot from the results from the first dataset, because of the selection bias in the first. Any effect observed was probably because of the characteristics of the different groups, and not because of the treatment. Because of the selection bias, we were trying to compare apples to oranges, while in the second data set, we were closer to comparing apples to apples. 

## Question 2

a) It is a question of causal inference because we care about the effect of Greta Thunberg on our outcome variable. We are thus interested in both our beta as well as our Y. 


b) The authors claim that this is about prediction, but if this was the case, we would only care about predicting the outcome, and not the specific effect of a certain treatment on the outcome. 


c) We think Clinton rushed her conclusion a bit, since there is probably some selection bias at play here. Having a high familiarity with Greta Thunberg is probably not random, so this effect could also be clouded by other factors that are not taken into account by the researchers.


d) As mentioned above, there are probably more factors involved in why you are familiar with Greta Thunberg than those that are included in the study, so the supposed effect of Greta on climate action might actually be because of something else. 
